# SIDIS cross section in TMD formalism

In this jupyter notebook we will calculate the SIDIS cross section in the TMD formalism for the process 
$\begin{equation}l(p_1) + N(p_2) \rightarrow l'(p_3) + h(p_4) + X\end{equation}$ where $l$ is a lepton, $N$ is a nucleon (proton), $h$ is a detected hadron and $X$ is an undetected hadronic system (i.e., the remnants). 

## Import required standard libraries

In [1]:
## -- Remember to use python@3.10 every time in the notebook LHAPDF is called and actively used.
## -- For just TMDs one can use the most recent python3 version.

# Print the python version
from platform import python_version
print(python_version())

3.10.17


In [2]:
# Import required libraries
import os
import yaml
import torch
import numpy as np
import lhapdf as lh
import matplotlib.pyplot as plt

# Import apfelpy
import apfelpy as ap

# Import costum modules
import modules.utilities as utl

## configuration map

The equivalent of the  `config.yaml` file in NangaParbat. This is the place where we define the inputs for the calculation, e.g., the perturbative order, the PDF set, the FF set, the scales, the kinematics grids, etc.

In [3]:
# Configuration
config = {
    # Perturbative order of the computation. 0: LL, 1: NLL, 2: NNLL, 3:
    # NNNLL, -1: NLL', -2: NNLL'
    "PerturbativeOrder": 3,

    # Collinear PDF set and member to be used
    "pdfset": {"name": "MMHT2014nnlo68cl", "member": 0},

    # Collinear FF set and member to be used
    "ffset": {"name": "MAPFF10NNLOPIp", "member": 0},

    # Initial and final scale-variation factors around mub = 2e^{-gamma_E}
    # / b and Q, respectively.
    "TMDscales": {"Ci": 1.0, "Cf": 1.0},

    # Reference value of alpha_em.
    "alphaem": {"aref": 0.0072973525693, "Qref": 91.1876, "run": True},

    # Parameters of the x-space subgrids on which collinear PDFs are
    # tabulated. The first element is the number of points, the second is
    # the initial x point and the third is the interpolation degree.
    "xgridpdf": [[100, 1e-4, 3], [60, 1e-1, 3], [50, 6e-1, 3], [50, 8e-1, 3]],

    # Parameters of the x-space subgrids on which collinear FFs are
    # tabulated.
    "xgridff": [[60, 1e-2, 3], [50, 6e-1, 3], [50, 8e-1, 3]],
    
    # Number of points, interpolation degree, and integration accuracy of
    # the grid in Q.
    "Qgrid": {"n" : 100, "InterDegree": 3, "eps" : 1e-3},

    # Number of points, interpolation degree, and integration accuracy of
    # the grid in Bjorken x (used for SIDIS).
    "xbgrid": {"n" : 100, "InterDegree": 2, "eps" : 1e-3},

    # Number of points, interpolation degree, and integration accuracy of
    # the grid in z (used for SIDIS).
    "zgrid": {"n" : 100, "InterDegree": 2, "eps" : 1e-3},
}


### Set qT cut

In [4]:
qToQcut = 0.3

### output folder

In [5]:
# Output folder -- DO NOT CREATE IT FOR NOW

output_folder = "_output"
# os.makedirs(output_folder, exist_ok=True)
# print(f"Creating folder '{output_folder}' to store the output.")

## PDF set

### Initialize PDF set

In [6]:
# PDF set configuration
pdf_name = config["pdfset"]["name"]
pdf_member = config["pdfset"]["member"]

# Initalise LHAPDF set
pdf = lh.mkPDF(pdf_name, pdf_member)  # type: ignore

# To find out which methods and attributes are available on the pdf object
# (from the lhapdf python interface), you can use the dir() function:
# print(dir(pdf))

LHAPDF 6.5.4 loading /opt/homebrew/Cellar/lhapdf/6.5.4/share/LHAPDF/MMHT2014nnlo68cl/MMHT2014nnlo68cl_0000.dat
MMHT2014nnlo68cl PDF set, member #0, version 3; LHAPDF ID = 25300


### Rotation in QCD evolution basis

Rotate the PDF set from the physics basis into the QCD evolution basis. The QCD evolution basis is where evolution happens with apfelxx. The physical basis refers to PDFs defined for each quark flavor, while the QCD evolution basis uses combinations of these PDFs that simplify the evolution equations by diagonalizing certain terms.

**Code Explanation**

A lambda function in Python is a small anonymous function defined with the lambda keyword. It can take any number of arguments but can only have one expression.

- `RotPDFs`: The name of the lambda function.
- `lambda x, mu`: Defines an anonymous function (lambda) that takes two arguments, x and mu.
- `ap.PhysToQCDEv(pdf.xfxQ(x, mu))`: The expression that the lambda function returns. It computes the PDFs at x and mu using pdf.xfxQ(x, mu) and then rotates them into the QCD evolution basis with ap.PhysToQCDEv.

If we rewrite the lambda function as a regular function for clarity, it would look like:

```python
def RotPDFs(x, mu):
    return ap.PhysToQCDEv(pdf.xfxQ(x, mu))
```


In [7]:
# Rotate PDF set into the QCD evolution basis
RotPDFs = lambda x, mu: ap.PhysToQCDEv(pdf.xfxQ(x, mu))

### Quark Thresholds and Masses

Get the heavy-quark thresholds from the PDF LHAPDF set. The equivalent of
    
```cpp
// Get heavy-quark thresholds from the PDF LHAPDF set
std::vector<double> Thresholds;
for (auto const& v : distpdf->flavors())
if (v > 0 && v < 7)
  Thresholds.push_back(distpdf->quarkThreshold(v));
```

In the python example in `apfelxx/pywrap/`, there is:

```python
# Vectors of masses and thresholds
Masses = [0, 0, 0, pdf.quarkThreshold(4), pdf.quarkThreshold(5)]
Thresholds = [0, 0, 0, pdf.quarkThreshold(4), pdf.quarkThreshold(5)]
```

In [8]:
# Get heavy-quark thresholds from the PDF LHAPDF set
Thresholds = []
for v in pdf.flavors():
    if v > 0 and v < 7:
        Thresholds.append(pdf.quarkThreshold(v))

# Now Thresholds contains the thresholds for quark flavors 1 to 6
print("Quark thresholds from the PDF set:", Thresholds)

# Masses of the quarks
Masses = [0, 0, 0, pdf.quarkThreshold(4), pdf.quarkThreshold(5)]

Quark thresholds from the PDF set: [0.0, 0.0, 0.0, 1.4, 4.75]


### apfelpy x-space grid for PDFs

In [9]:
# Setup APFEL++ x-space grid for PDFs
gpdf = ap.Grid([ap.SubGrid(*subgrids) for subgrids in config["xgridpdf"]])

# Check
gxb = ap.Grid([ap.SubGrid(100, 1e-4, 3), ap.SubGrid(60,1e-1,3), ap.SubGrid(50,6e-1,3), ap.SubGrid(50,8e-1,3)])
if gpdf == gxb:
    print("Grids are the same.")
else:
    print("Grids are different.")

print("x-space grid:")
# help(ap.Grid)
# print(dir(g))
gpdf.Print()

Grids are the same.
x-space grid:
Grid: 0x11aade5c0
JointGrid = 0x11aade620
0.0001 0.000109648 0.000120226 0.000131826 0.000144544 0.000158489 0.00017378 0.000190546 0.00020893 0.000229087 0.000251189 0.000275423 0.000301995 0.000331131 0.000363078 0.000398107 0.000436516 0.00047863 0.000524807 0.00057544 0.000630957 0.000691831 0.000758578 0.000831764 0.000912011 0.001 0.00109648 0.00120226 0.00131826 0.00144544 0.00158489 0.0017378 0.00190546 0.0020893 0.00229087 0.00251189 0.00275423 0.00301995 0.00331131 0.00363078 0.00398107 0.00436516 0.0047863 0.00524807 0.0057544 0.00630957 0.00691831 0.00758578 0.00831764 0.00912011 0.01 0.0109648 0.0120226 0.0131826 0.0144544 0.0158489 0.017378 0.0190546 0.020893 0.0229087 0.0251189 0.0275423 0.0301995 0.0331131 0.0363078 0.0398107 0.0436516 0.047863 0.0524807 0.057544 0.0630957 0.0691831 0.0758578 0.0831764 0.0912011 0.1 0.104713 0.109648 0.114815 0.120226 0.125893 0.131826 0.138038 0.144544 0.151356 0.158489 0.165959 0.17378 0.18197 0.19054

### Perturbative order


In [10]:
PerturbativeOrder = config["PerturbativeOrder"]

## AlphaS

Get the strong coupling constant $\alpha_s$ from the PDF LHAPDF set. The equivalent of the following C++ code, which is used in NangaParbat.

```cpp
// Alpha_s (from PDFs). Get it from the LHAPDF set and tabulate it.
  const auto Alphas = [&] (double const& mu) -> double{ return distpdf->alphasQ(mu); };
  const apfel::TabulateObject<double> TabAlphas {[&] (double const& mu) -> double{return distpdf->alphasQ(mu); },
                                                100, distpdf->qMin() * 0.9, distpdf->qMax(), 3, Thresholds};
```

Note that the C++ code in NangaParbat uses the LHAPDF's alphasQ(mu) function for the running coupling $\alpha_s(\mu)$, and then tabulating it with `TabulateObject`.
The equivalent of this C++ implementation is implemented below in this notebook. Pros:

- Ensures that $\alpha_s(\mu)$ used in your calculations exactly matches that from the PDF set.
- Avoids discrepancies due to differences in perturbative orders, thresholds, or initial conditions.
- Mimics the C++ code where `alphasQ(mu)` from `LHAPDF` is directly used.

**Alternative python implementation that uses `APFEL++`'s internal `AlphaQCD` class to compute the running coupling $\alpha_s(\mu)$:**
In the python example in `apfelxx/pywrap/`, there is:

```python
# Get perturbative order from LHAPDF set
PerturbativeOrder = pdf.orderQCD

# Running coupling
Alphas = ap.AlphaQCD(
    pdf.alphasQ(ap.constants.ZMass),  # Alpha_s at reference scale
    ap.constants.ZMass,               # Reference scale (Z boson mass)
    Thresholds,                       # Quark mass thresholds
    PerturbativeOrder                 # Perturbative order
)
TabAlphas = ap.TabulateObject(a, 100, 0.9, 1001, 3)
```

In this python implementation, we are using `APFEL++`'s internal `AlphaQCD` class to compute the running coupling $\alpha_s(\mu)$, initialized with parameters from the LHAPDF set. Pros:

- Leverages `APFEL++`'s internal implementation of the running coupling.
- You can specify the perturbative order and thresholds explicitly.
- Consistent with `APFEL++`'s evolution mechanisms.

Cons:

- May lead to inconsistencies if the parameters (e.g., initial $\alpha_s$, perturbative order, thresholds) do not exactly match those used in the PDF set.
- Requires careful synchronization of parameters to ensure consistency.

In [11]:
# Define the Alpha_s function using the LHAPDF set
Alphas = lambda mu: pdf.alphasQ(mu)

# Create a TabulateObject for Alpha_s
TabAlphas = ap.TabulateObject(
    Alphas,
    100,                         # Number of points in the mu grid
    np.sqrt(pdf.q2Min) * 0.9,    # Minimum mu value (QMin)
    np.sqrt(pdf.q2Max),          # Maximum mu value (QMax)
    3,                           # Interpolation degree
    Thresholds                   # Quark mass thresholds
)

Tabulating object... Time elapsed: 0.000133 seconds


## Scales

In [12]:
# Get scale-variation factors
Ci = config["TMDscales"]["Ci"]
Cf = config["TMDscales"]["Cf"]

# Initial scale
mu0 = np.sqrt(pdf.q2Min)

## Electromagnetic coupling squared (provided by APFEL++)

The following apfelpy code is the equivalent of the C++ code in NangaParbat that computes the electromagnetic coupling squared $\alpha_{\text{em}}^2$.

```cpp
  const double aref = config["alphaem"]["aref"].as<double>();
  apfel::AlphaQED alphaem{aref, 
                          config["alphaem"]["Qref"].as<double>(), 
                          Thresholds, 
                          {0, 0, 1.777}, 
                          0};
  const apfel::TabulateObject<double> TabAlphaem{alphaem, 100, 0.9, 1001, 3};
 ``` 

In [13]:
# Extract 'aref' and 'Qref' from the configuration
aref = config["alphaem"]["aref"]
Qref = config["alphaem"]["Qref"]

# Lepton mass thresholds (electron, muon, tau masses in GeV).
# Electron and muon masses are approximately zero compared to the tau mass
LeptThresholds = [0.0, 0.0, 1.777]

# Quark mass thresholds (previously defined)
# Assuming 'Thresholds' is already defined in your code
QuarkThresholds = Thresholds

# Perturbative order (0 for Leading Order)
pto_aem = 0


# Initialize the AlphaQED object
alphaem = ap.AlphaQED(
    AlphaRef=aref,
    MuRef=Qref,
    LeptThresholds=LeptThresholds,
    QuarkThresholds=QuarkThresholds,
    pt=pto_aem                            # Perturbative order
)

# Create a TabulateObject for alphaem
TabAlphaem = ap.TabulateObject(
    alphaem,
    100,     # Number of points in the mu grid
    0.9,     # Minimum mu value
    1001,    # Maximum mu value
    3        # Interpolation degree
)


Tabulating object... Time elapsed: 0.000296 seconds


## Evolve PDFs

Construct set of distributions as a function of the scale to be tabulated. The equivalent of the following C++ code in NangaParbat.

```cpp
  const auto EvolvedPDFs = [=,&gpdf] (double const& mu) -> apfel::Set<apfel::Distribution>
  {
    return apfel::Set<apfel::Distribution>{apfel::EvolutionBasisQCD{apfel::NF(mu, Thresholds)}, DistributionMap(gpdf, RotPDFs, mu)};
  };

  // Tabulate collinear PDFs
  const apfel::TabulateObject<apfel::Set<apfel::Distribution>> TabPDFs{EvolvedPDFs, 100, distpdf->qMin() * 0.9, distpdf->qMax(), 3, Thresholds};
  const auto CollPDFs = [&] (double const& mu) -> apfel::Set<apfel::Distribution> { return TabPDFs.Evaluate(mu); };
```

In [14]:
### --- This is a working code coming from ApfelPy_example.ipynb 

### --- Check if it does exactly the same things as the c++ lines reported above from the 
###     NangaParbat file tests/SIDISMultiplicities.cc --- ###

# Initialize QCD evolution objects
DglapObj = ap.initializers.InitializeDglapObjectsQCD(gpdf, Masses, Thresholds)

# Construct the DGLAP objects
EvolvedPDFs = ap.builders.BuildDglap(DglapObj, lambda x, mu: ap.utilities.PhysToQCDEv(pdf.xfxQ(x, mu)), mu0, pdf.orderQCD, TabAlphas.Evaluate)

# Tabulate collinear PDFs
# TabPDFs = ap.TabulateObjectSetD(EvolvedPDFs, 100, 1, 1000, 3)
TabPDFs = ap.TabulateObjectSetD(EvolvedPDFs, 100, np.sqrt(pdf.q2Min) * 0.9, np.sqrt(pdf.q2Max), 3)
CollPDFs = lambda mu: TabPDFs.Evaluate(mu)

Initializing DglapObjects for space-like QCD unpolarised evolution... Time elapsed: 0.052271 seconds
Tabulating object... Time elapsed: 0.082036 seconds


## TMDs

In [15]:
from modules.fnp import fNP

## Initialize fNP

In [16]:
# Load the configuration from a YAML file

config_file_path = 'inputs/fNPconfig.yaml'
config_fnp = utl.load_yaml_file(config_file_path)


In [17]:
# Instantiate the model.
# The object model_fNP itself is not just a dictionary; 
# it is an nn.Module that contains an nn.ModuleDict as one of its attributes.
model_fNP = fNP(config_fnp)
   

[fNPManager] Initializing PDF and FF fNP modules
[fNPManager] Setting up shared evolution: g2=0.1284, trainable=True
[fNPManager] Initialized 8 PDF flavor modules
[fNPManager] Initialized 8 FF flavor modules
✅ fNP manager initialization completed



## TMD objects

In [18]:
# Initialize TMD objects
TmdObjPDF = ap.tmd.InitializeTmdObjects(gpdf, Thresholds)

Initializing DglapObjects for space-like QCD unpolarised evolution... Time elapsed: 0.052617 seconds
Initializing DglapObjects for time-like QCD unpolarised evolution... Time elapsed: 0.026816 seconds
Initializing TMD objects for matching and evolution... Time elapsed: 0.053125 seconds


In [19]:
# Build evolved TMD PDFs
EvTMDPDFs    = ap.tmd.BuildTmdPDFs(TmdObjPDF, CollPDFs, Alphas, PerturbativeOrder, Ci);
MatchTMDPDFs = ap.tmd.MatchTmdPDFs(TmdObjPDF, CollPDFs, Alphas, PerturbativeOrder, Ci);

In [20]:
QuarkSudakov = ap.tmd.QuarkEvolutionFactor(TmdObjPDF, Alphas, PerturbativeOrder, Ci, 1e5);

# Get hard-factor
Hf = ap.tmd.HardFactor("SIDIS", TmdObjPDF, Alphas, PerturbativeOrder, Cf);

## Data - kinematics

In [21]:
### --- Example of data file

# NOTE: dsigma here is the multiplicity, here I copied a real HERMES bin: 
#       HERMES_Pro_Pip_x_0.023_0.047_z_0.475_0.6.yaml

dataset = {
    "header": {
        "process": "SIDIS",
        "observable": "multiplicity",
        "target_isoscalarity": 1,
        "hadron": "PI",
        "charge": +1,
        "Vs": 7.2565449,
        "PS_reduction": {
            "W": 3.1622777,
            "ymin": 0.1,
            "ymax": 0.85
        },  
    },
    "data": {
        "PhT": [0.1030419, 0.2055688, 0.3056045, 0.4083209, 0.5321106, 0.7035105, 0.9611047],
        "x":   [0.03758844, 0.03758844, 0.03758844, 0.03758844, 0.03758844, 0.03758844, 0.03758844],
        "z":   [0.5334359, 0.5370032, 0.5377358, 0.5421495, 0.5419208, 0.5442552, 0.5456988],
        "Q2":  [1.249727, 1.249727, 1.249727, 1.249727, 1.249727, 1.249727, 1.249727],
        "y" :  [0.63139491, 0.63139491, 0.63139491, 0.63139491, 0.63139491, 0.63139491, 0.63139491],
        "dsigma": [0.1864647, 0.4160929, 0.5467943, 0.5739435, 0.5183662, 0.3815261, 0.1247706],
    },
    
    
}

# Example usage:
print(dataset["header"]["hadron"])
print(dataset["header"]["charge"])
print(dataset["data"]["PhT"])

PI
1
[0.1030419, 0.2055688, 0.3056045, 0.4083209, 0.5321106, 0.7035105, 0.9611047]


# Initialize collinear FF set

In [22]:
# PDF set configuration
ff_name = config["ffset"]["name"]
ff_member = config["ffset"]["member"]

# Initalise LHAPDF set
distff = lh.mkPDF(ff_name, ff_member) # type: ignore

LHAPDF 6.5.4 loading /opt/homebrew/Cellar/lhapdf/6.5.4/share/LHAPDF/MAPFF10NNLOPIp/MAPFF10NNLOPIp_0000.dat
MAPFF10NNLOPIp PDF set, member #0, version 1; LHAPDF ID = 2021000


## Rotation of collinear FF set in QCD evolution basis

In [23]:
# Rotate PF set into the QCD evolution basis
RotFFs = lambda x, mu: ap.PhysToQCDEv(distff.xfxQ(x, mu))

### apfelpy x-space grid for collinear FFs

In [24]:
# Setup APFEL++ x-space grid for FFs
gff = ap.Grid([ap.SubGrid(*subgrids) for subgrids in config["xgridff"]])

print("x-space grid:")
# help(ap.Grid)
# print(dir(g))
gff.Print()

x-space grid:
Grid: 0x11c9fb150
JointGrid = 0x11c9fb1b0
0.01 0.0107978 0.0116591 0.0125893 0.0135936 0.014678 0.0158489 0.0171133 0.0184785 0.0199526 0.0215443 0.0232631 0.0251189 0.0271227 0.0292864 0.0316228 0.0341455 0.0368695 0.0398107 0.0429866 0.0464159 0.0501187 0.054117 0.0584341 0.0630957 0.0681292 0.0735642 0.0794328 0.0857696 0.0926119 0.1 0.107978 0.116591 0.125893 0.135936 0.14678 0.158489 0.171133 0.184785 0.199526 0.215443 0.232631 0.251189 0.271227 0.292864 0.316228 0.341455 0.368695 0.398107 0.429866 0.464159 0.501187 0.54117 0.584341 0.630957 0.63704 0.643181 0.649382 0.655642 0.661962 0.668344 0.674787 0.681292 0.68786 0.694491 0.701186 0.707946 0.714771 0.721661 0.728618 0.735642 0.742734 0.749894 0.757123 0.764422 0.771792 0.779232 0.786744 0.794328 0.801986 0.805842 0.809717 0.813611 0.817523 0.821454 0.825404 0.829373 0.833361 0.837369 0.841395 0.845441 0.849506 0.853591 0.857696 0.86182 0.865964 0.870128 0.874312 0.878517 0.882741 0.886986 0.891251 0.895537 0.89

## Evolve collinear FFs

This is the equivalent of the C++ code:

```cpp
// Construct set of distributions as a function of the scale to be
// tabulated
const auto EvolvedFFs = [=,&gff] (double const& mu) -> apfel::Set<apfel::Distribution>
{
    return apfel::Set<apfel::Distribution>{apfel::EvolutionBasisQCD{apfel::NF(mu, Thresholds)}, DistributionMap(gff, RotFFs, mu)};
};

// Tabulate collinear FFs
const apfel::TabulateObject<apfel::Set<apfel::Distribution>> TabFFs{EvolvedFFs, 200, distff->qMin() * 0.9, distff->qMax(), 3, Thresholds};
const auto CollFFs = [&] (double const& mu) -> apfel::Set<apfel::Distribution> { return TabFFs.Evaluate(mu); };

```

In [25]:
# Construct set of distributions as a function of the scale to be tabulated

# Initialize QCD evolution objects
DglapObjFF = ap.initializers.InitializeDglapObjectsQCD(gff, Masses, Thresholds)

# Construct the DGLAP objects
EvolvedFFs = ap.builders.BuildDglap(DglapObjFF, lambda x, mu: ap.utilities.PhysToQCDEv(distff.xfxQ(x, mu)), mu0, distff.orderQCD, TabAlphas.Evaluate)

# Tabulate collinear FFs
TabFFs = ap.TabulateObjectSetD(EvolvedFFs, 100, np.sqrt(distff.q2Min) * 0.9, np.sqrt(distff.q2Max), 3)
CollFFs = lambda mu: TabFFs.Evaluate(mu)

Initializing DglapObjects for space-like QCD unpolarised evolution... Time elapsed: 0.035947 seconds
Tabulating object... Time elapsed: 0.060140 seconds


## TMD FFs Objects

In [26]:
# Initialize TMD FF objects
TmdObjFF  = ap.tmd.InitializeTmdObjects(gff, Thresholds);

Initializing DglapObjects for space-like QCD unpolarised evolution... Time elapsed: 0.035401 seconds
Initializing DglapObjects for time-like QCD unpolarised evolution... Time elapsed: 0.018379 seconds
Initializing TMD objects for matching and evolution... Time elapsed: 0.037038 seconds


In [27]:
# Build evolved TMD FFs
EvTMDFFs    = ap.tmd.BuildTmdFFs(TmdObjFF, CollFFs, Alphas, PerturbativeOrder, Ci);
MatchTMDFFs = ap.tmd.MatchTmdFFs(TmdObjFF, CollFFs, Alphas, PerturbativeOrder, Ci);

# Kinematics

## Get kinematics

In [28]:
# Get kinematic variables from the dataset

# Scalars (we can leave these as Python numbers, or wrap as 0-D tensors)
Vs_aslist        = dataset["header"]["Vs"]
targetiso_aslist = dataset["header"]["target_isoscalarity"]

# Grab raw data (list or np.array)
x_np    = dataset["data"]["x"]
Q2_np   = dataset["data"]["Q2"]
z_np    = dataset["data"]["z"]
PhT_np  = dataset["data"]["PhT"]



## Convert kinematics to torch tensors

In [29]:
# Convert to torch tensors (choose dtype and, if you want, move to GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype  = torch.float32

Vs        = torch.tensor(dataset["header"]["Vs"], dtype=torch.float32, device=device)
targetiso = torch.tensor(dataset["header"]["target_isoscalarity"], dtype=torch.float32, device=device)

x_tens    = torch.tensor(x_np,    dtype=dtype, device=device)
Q2_tens   = torch.tensor(Q2_np,   dtype=dtype, device=device)
z_tens    = torch.tensor(z_np,    dtype=dtype, device=device)
PhT_tens  = torch.tensor(PhT_np,  dtype=dtype, device=device)

# Compute what is needed with pure torch ops
Q_tens   = torch.sqrt(Q2_tens)         # elementwise √
qT_tens  = PhT_tens / z_tens           # elementwise division


In [30]:
# Check 
print("x:", x_tens)
print("Q:", Q_tens)
print("z:", z_tens)
print("PhT:", PhT_tens)
print("qT:", qT_tens)

# Check the length of the vectors
print("Length of x:", len(x_tens))
print("Length of Q:", len(Q_tens))
print("Length of z:", len(z_tens))
print("Length of PhT:", len(PhT_tens))

x: tensor([0.0376, 0.0376, 0.0376, 0.0376, 0.0376, 0.0376, 0.0376])
Q: tensor([1.1179, 1.1179, 1.1179, 1.1179, 1.1179, 1.1179, 1.1179])
z: tensor([0.5334, 0.5370, 0.5377, 0.5421, 0.5419, 0.5443, 0.5457])
PhT: tensor([0.1030, 0.2056, 0.3056, 0.4083, 0.5321, 0.7035, 0.9611])
qT: tensor([0.1932, 0.3828, 0.5683, 0.7532, 0.9819, 1.2926, 1.7612])
Length of x: 7
Length of Q: 7
Length of z: 7
Length of PhT: 7


In [31]:
### --- Taking into account the isoscalarity of the target

# sign = +1 where targetiso >= 0, else -1
sign = torch.where(targetiso >= 0,
                   torch.ones_like(targetiso),
                   -torch.ones_like(targetiso))

# Fraction of protons = |targetiso|
frp = targetiso.abs()

# Fraction of neutrons = 1 − frp
frn = 1 - frp

# Tabulate initial scale TMD PDFs

This is the implementation of:

```cpp
// Tabulate initial scale TMD PDFs in b in the physical basis
// taking into account the isoscalarity of the target.
std::function<apfel::Set<apfel::Distribution>(double const&)> isTMDPDFs =
    [&] (double const& b) -> apfel::Set<apfel::Distribution>
{
    const apfel::Set<apfel::Distribution> xF = QCDEvToPhys(MatchTMDPDFs(b).GetObjects());
    std::map<int, apfel::Distribution> xFiso;

    // Treat down and up separately to take isoscalarity of
    // the target into account.
    xFiso.insert({1,  frp * xF.at(sign) + frn * xF.at(sign*2)});
    xFiso.insert({-1, frp * xF.at(-sign) + frn * xF.at(-sign*2)});
    xFiso.insert({2,  frp * xF.at(sign*2) + frn * xF.at(sign)});
    xFiso.insert({-2, frp * xF.at(-sign*2) + frn * xF.at(-sign)});
    
    // Now run over the remaining flavours
    for (int i = 3; i <= 6; i++)
    {
        const int ip = i * sign;
        xFiso.insert({i, xF.at(ip)});
        xFiso.insert({-i, xF.at(-ip)});
    }
    return apfel::Set<apfel::Distribution>{xFiso};
};
const apfel::TabulateObject<apfel::Set<apfel::Distribution>> TabMatchTMDPDFs{isTMDPDFs, 200, 1e-2, 2, 1, {},
                                                                                [] (double const& x) -> double{ return log(x); },
                                                                                [] (double const& y) -> double{ return exp(y); }};

```

In [32]:
# Functions used for the tabulation
# Use a 64-bit float tensor so you don’t lose precision
def TabFunc(b: float) -> float:
    t = torch.tensor(b, dtype=torch.float64)
    return t.log().item()

def InvTabFunc(y: float) -> float:
    t = torch.tensor(y, dtype=torch.float64)
    return t.exp().item()

In [33]:
# This is the python version of the C++ lambda isTMDPDFs
def isTMDPDFs(b):
    """
    b: impact-parameter (float)
    returns: Set<Distribution> in the physical basis,
             with isoscalarity built in.
    """

    # Match in b-space, then rotate back to physical basis.
    # The b dependence in this case is hidden in the TMD object, TmdObjpDF 
    # (which internally knows how to carry the b–dependence).
    # matched = ap.tmd.MatchTmdPDFs(TmdObjPDF, CollPDFs, Alphas, PerturbativeOrder, Ci)                                                 
    # xF      = ap.utilities.QCDEvToPhys(matched.GetObjects())
    
    # This does the same thing of the two lines above, but in just one line, 
    # MatchTMDPDFs has already been defined above, here is just reported 
    # (tecnically is re-defined in teh same exact way)
    MatchTMDPDFs = ap.tmd.MatchTmdPDFs(TmdObjPDF, CollPDFs, Alphas, PerturbativeOrder, Ci);
    xF = ap.utilities.QCDEvToPhys(MatchTMDPDFs(b).GetObjects()) # now xF is a dict keyed by ints: { …, 1: Distribution, 2: Distribution, …}

    # Deal with the fact that sign is a pytorch tensor, not a number.
    # Extract your integer keys from the tensor
    s    = int(sign.item())        # e.g. +1 or -1
    s2   = int((sign * 2).item())  # +2 or -2

    # Build the isoscalar map.
    # Treat down and up separately to take isoscalarity of
    # the target into account.    
    # Build the isoscalar map, but keep frp/frn as tensors
    xFiso = {}
    xFiso[ 1] = frp * xF[  s] + frn * xF[  s2]
    xFiso[-1] = frp * xF[ -s] + frn * xF[ -s2]
    xFiso[ 2] = frp * xF[  s2] + frn * xF[  s]
    xFiso[-2] = frp * xF[ -s2] + frn * xF[ -s]
    
    # Now run over the remaining flavors
    for i in range(3,7):
        ip = int((i * sign).item())    # e.g. 3→3 or 3→-3
        xFiso[ i] = xF[ip]
        xFiso[-i] = xF[-ip]

    return ap.SetD(xFiso)  # wrap into the Python Set<Distribution> 


In [34]:
TabMatchTMDPDFs = ap.TabulateObjectSetD(
    isTMDPDFs,    # function(double)->SetD
    200,          # nQ points
    1e-2,         # QMin
    2,            # QMax
    1,            # interpolation degree
    [],           # empty thresholds vector {}
    TabFunc,      # log tabulation
    InvTabFunc    # exp inverse
)  

Tabulating object... Time elapsed: 0.514774 seconds


# Tabulate initial scale TMD FFs in b in the physical basis

In [35]:
isTMDFFs = lambda b: ap.SetD(
    ap.utilities.QCDEvToPhys(
        MatchTMDFFs(b).GetObjects()
    )
)

In [36]:
TabMatchTMDFFs = ap.TabulateObjectSetD(
    isTMDFFs,    # function(double)->SetD
    200,          # nQ points
    1e-2,         # QMin
    2,            # QMax
    1,            # interpolation degree
    [],           # empty thresholds vector {}
    TabFunc,      # log tabulation
    InvTabFunc    # exp inverse
)  

Tabulating object... Time elapsed: 0.077257 seconds


# Compute differential cross sections

begins l.361 of SIDISMultiplicities.cc

In [37]:
# Implementation of the bstar prescription
# Since NangaParbat's bstar functions are not available in the Python wrapper,
# we implement a simple bstar_min prescription

def bstar_min(b, Q):
    """
    Simple bstar prescription (bstar_min variant)
    b: impact parameter (float)
    Q: hard scale (float)
    """
    bmax = 1.5  # GeV^-1, typical value used in TMD phenomenology
    gamma_E = 0.5772156649015329  # Euler-Mascheroni constant
    return b / np.sqrt(1 + (b / bmax)**2) * 2 * np.exp(-gamma_E) / Q

In [38]:
# Initialize missing objects before computation

# Initialize Ogata quadrature object for b-space integration
DEObj = ap.ogata.OgataQuadrature(0, 1e-9, 0.00001)  # (order, cutoff, step)

# Initialize results tensor
theo_xsec = torch.zeros_like(qT_tens)

print(f"Starting SIDIS cross section computation for {len(qT_tens)} qT values...")

# Computation loop for each qT value
for iqT in range(len(qT_tens)):
    # Convert to float for APFEL++ compatibility
    qTm = float(qT_tens[iqT].item())
    Qm  = float(Q_tens[iqT].item()) 
    xm  = float(x_tens[iqT].item())
    zm  = float(z_tens[iqT].item())
    
    # Do not compute if qT > cut * Q
    if qTm > qToQcut * Qm:
        print(f"Skipping qT = {qTm:.3f} (above cut)")
        continue
    
    print(f"Computing qT = {qTm:.3f}, Q = {Qm:.3f}, x = {xm:.4f}, z = {zm:.4f}")
    
    # Scales
    mu = Cf * Qm
    zeta = Qm * Qm
    
    # Yp factor for SIDIS kinematics 
    Yp = 1 + (1 - (Qm / Vs)**2 / xm)**2
    
    # Electroweak charges (only photon contribution)
    # Number of active flavors
    nf = int(ap.utilities.NF(mu, Thresholds))
    
    print(f"  Scales: mu = {mu:.3f}, zeta = {zeta:.3f}")
    print(f"  Active flavors: {nf}, Yp = {Yp:.3f}")
    
    # Define b-integrand function for Ogata quadrature
    def b_integrand(b):
        # bstar prescription
        bs = bstar_min(b, Qm)
        
        # TMD luminosity: sum over active quark flavors
        lumiq = 0.0
        for q in range(-nf, nf+1):
            if q == 0:  # Skip gluon
                continue
                
            # Get TMD PDF at (q, x, bs)
            tmd_pdf = TabMatchTMDPDFs.EvaluatexQ(q, xm, bs)
            
            # Get TMD FF at (q, z, bs) 
            tmd_ff = TabMatchTMDFFs.EvaluatexQ(q, zm, bs)
            
            # Electric charge squared for this quark flavor
            qch2 = ap.constants.QCh2[abs(q)-1] if abs(q) <= len(ap.constants.QCh2) else 0.0
            
            # Luminosity contribution
            lumiq += Yp * tmd_pdf / xm * qch2 * tmd_ff
        
        # Non-perturbative evolution factors (using fNP model)
        # For now, use a simple Gaussian form as placeholder
        # TODO: Replace with actual fNP model evaluation
        fnp1 = np.exp(-0.1 * b**2)  # PDF non-perturbative function
        fnp2 = np.exp(-0.05 * b**2)  # FF non-perturbative function
        
        # Sudakov factor
        sudakov_factor = QuarkSudakov(bs, mu, zeta)**2
        
        # Hard factor
        hard_factor = Hf(mu)
        
        # Alpha_em^2 factor
        alphaem2 = TabAlphaem.Evaluate(Qm)**2
        
        # Full integrand
        integrand = (b * fnp1 * fnp2 * lumiq * sudakov_factor * 
                    alphaem2 * hard_factor / (Qm**3 * zm))
        
        return integrand
    
    # Perform b-space integration using Ogata quadrature
    try:
        integral_result = DEObj.transform(b_integrand, qTm)
        
        # Differential cross section (numerator only, no denominator)
        differential_xsec = (ap.constants.ConvFact * ap.constants.FourPi * 
                           qTm * integral_result / (2 * Qm) / zm)
        
        theo_xsec[iqT] = differential_xsec
        
        print(f"  Integration successful: σ = {differential_xsec:.6e}")
        
    except Exception as e:
        print(f"  Integration failed: {e}")
        theo_xsec[iqT] = 0.0

print("SIDIS cross section computation completed!")


Starting SIDIS cross section computation for 7 qT values...
Computing qT = 0.193, Q = 1.118, x = 0.0376, z = 0.5334
  Scales: mu = 1.118, zeta = 1.250
  Active flavors: 3, Yp = 1.136
  Integration successful: σ = 1.897599e+04
Skipping qT = 0.383 (above cut)
Skipping qT = 0.568 (above cut)
Skipping qT = 0.753 (above cut)
Skipping qT = 0.982 (above cut)
Skipping qT = 1.293 (above cut)
Skipping qT = 1.761 (above cut)
SIDIS cross section computation completed!
